In [19]:
import pandas as pd
import numpy as np
np.random.seed(42)

In [20]:
df = pd.util.testing.makeTimeDataFrame(5)

In [21]:
df

,A,B,C,D
2000-01-03,0.496714,-0.234137,-0.463418,-0.562288
2000-01-04,-0.138264,1.579213,-0.465730,-1.012831
2000-01-05,0.647689,0.767435,0.241962,0.314247
2000-01-06,1.523030,-0.469474,-1.913280,-0.908024
2000-01-07,-0.234153,0.542560,-1.724918,-1.412304


In [22]:
df.to_csv('0323.csv')

预期结果：

第一行为：

In [23]:
result_1 = df.iloc[:4].corr(method='spearman').mean()
result_1.name = df.index[3]
result_1 = result_1.to_frame().T
result_1

,A,B,C,D
2000-01-06,0.1,0.1,0.5,0.5


第二行为：

In [24]:
result_2 = df.iloc[1:5].corr(method='spearman').mean()
result_2.name = df.index[4]
result_2 = result_2.to_frame().T
result_2

,A,B,C,D
2000-01-07,0.3,0.35,0.5,0.55


In [29]:
result = pd.concat([result_1, result_2])

In [ ]:
result.to_csv('0323_结果.csv')